In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

startDate = '20201015'
endDate = '20201125'


readPath = r'\\192.168.10.28\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
rawOrderLog = []
for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_csv(thisPath)
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

display('There are accounts with duplicated ticks:')
display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

# assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
#                                                    .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
#                                                                 'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
# if rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
#                                                    .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
#                                                                 'absOrderSize', 'internalId'], keep=False)].shape[0] != 0:
#     assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
#                                                    .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
#                                                                 'absOrderSize', 'internalId'], keep=False)]['accCode'].unique() == [8856])
#     print('8856 has wrong internalId')
#     assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
#             'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0)
#     if rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['vai'] == -1)].shape[0] != 0:
#         print(rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['vai'] == -1)][['date', 'colo', 'accCode', \
#             'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])   
# else:
#     print('we can use internalId in 8856')
#     assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
#             'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0)   

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
    
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog["broker"])
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
# rawOrderLog = rawOrderLog[rawOrderLog['ars'] % 10 == 1]


### Assertion 1:  make sure same direction in same date, secid, vai
print('=======================================================================================')
print('1. same date, secid, vai: same direction')
orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
if len(orderLog[orderLog['directNum'] != 1]) > 0:
    print('opposite direction for same date, same secid, same vai')
    display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
    orderLog = orderLog[orderLog['directNum'] == 1]

assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

## Assertion 2:  make sure each account, secid, vai only has one insertion
print('=======================================================================================')
print('2. same date, secid, vai, accCode: one insertion')
a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
if len(a[a > 1]) > 0:
    print('more than one insertion at same time')
    a = a[a>1].reset_index()
    display(a)
    display(a.groupby(['accCode'])['date'].size())
    orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


### Assertion 3:  check IPO stocks selling status
print('=======================================================================================')
print('3. IPO stocks selling (ars = 301, 302)')
if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
    kk = orderLog[orderLog['ars'].isin([301, 302])]
    print(kk)
    try:
        assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
        print('we only sell, never buy')
    except:
        print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk[kk['orderDirection1'] == 1])
    kk1 = kk[kk['updateType'] == 0]
    kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
    kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
    kk1['diff'] = kk1['diff'].fillna(0)
    try:
        assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
        print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
    except:
        print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk1[kk1['diff'] < 10e6])
    kk2 = kk[(kk['updateType'] == 1)]
    try:
        assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
        print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
    except:
        print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(kk2[kk2['duration'] < 3e6])


### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
print('=======================================================================================')
print('4. updateType 7 orders')
if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
    assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
print('=======================================================================================')
print('5. updateType 6 orders')
k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
k['prob'] = k['order_x']/k['order_y']
try:
    assert(sum(k['prob'] >= 0.05) == 0)
except:
    print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    print(k[k['prob'] >= 0.05])

### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
print('=======================================================================================')
print('6. CYB stocks order size < 30w')
try:
    cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
    assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
except:
    print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
### Assertion 7:  make sure there is no unexpected updateType 
print('=======================================================================================')
print('7. unexpected updateType')
def getTuple(x):
    return tuple(i for i in x)

checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                               (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                               (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                     np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

### Assertion 8:  make sure status==0 got all traded
print('=======================================================================================')
print('8. status == 0: all traded')
a = orderLog[orderLog['status'] == 0]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in total trade, any fill != total cases')
display(a[a['filled'] != a['total']])
if a[a['filled'] != a['total']].shape[0] > 0:
    removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 9:  make sure status==1 got partial traded
print('=======================================================================================')
print('9. status == 1: partial traded')
a = orderLog[orderLog['status'] == 1]
a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
a.columns = ['order', 'filled', 'total']
print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
    removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
### Assertion 10: make sure no cancellation within 1 sec
print('=======================================================================================')
print('10. no cancellation within 1 sec')
a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
print('any cancellation within 1 sec')
display(a)
if a.shape[0] > 0:
    removeOrderLs = a['order'].unique()
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


### Assertion 11: make sure no order has shares > 80w or notional > 800w
print('=======================================================================================')
print('11. Orders with size > 80w or notional > 800w')
orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
    print('some order quantity are > 80w')
    print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection', 'clock', 'order']])
            
if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
    print('some order ntl are > 800w')
    print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
    display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                      'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                      "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['mrsb90'] == '-'])
orderLog = orderLog[orderLog['mrsb90'] != '-']
orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
    display(orderLog[orderLog['aaa'] == '-'])
orderLog = orderLog[orderLog['aaa'] != '-']
orderLog['aaa'] = orderLog['aaa'].astype(float)
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] >= 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
orderLog.loc[(orderLog['orderDirection'] < 1) &\
         (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
orderLog = orderLog[~orderLog['ars'].isnull()]
orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
orderLog['mrstaat'] == 3000, '2. statwo', np.where(
orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (50,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (34,50,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (51,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\U

'There are accounts with duplicated ticks:'

date      colo      accCode
20201106  zs_88_04  892402     11272
          zs_96_02  9655        1450
          zt_88_02  897102     20282
20201112  zs_96_08  974101     27542
          zt_88_06  8971       25984
Name: ars, dtype: int64

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
347326,20201015,zs_52_09,5291,2000690,-1,7,34779.0,0,0,-1.0,-1.000000e+00
355505,20201015,zs_52_06,5269,2000762,-1,1,47723.0,0,0,-1.0,1.312819e+06
380405,20201015,zs_52_09,5291,2000998,-1,1,39199.0,0,0,-1.0,1.040596e+06
405206,20201015,zs_64_01,6480,2002098,-1,1,47430.0,0,0,-1.0,3.641870e+05
414924,20201015,zs_52_08,5281,2002153,-1,1,40041.0,0,0,-1.0,1.120545e+06
...,...,...,...,...,...,...,...,...,...,...,...
28304713,20201125,zs_66_01,6634,2300642,-1,7,36827.0,0,0,-1.0,-1.000000e+00
28317059,20201125,zs_52_06,5287,2300674,-1,1,47824.0,0,0,-1.0,1.103285e+06
28325666,20201125,zs_54_01,5474,2300693,-1,1,36051.0,0,0,-1.0,8.689017e+08
28340274,20201125,zs_96_08,9685,2300724,-1,7,52554.0,0,0,-1.0,7.221449e+17


...

,date,colo,accCode,secid,vai,order,clockAtArrival
0,20201015,zs_54_01,5456,2002947,184300.0,67411,2
1,20201015,zs_54_01,5456,2300552,38000.0,67835,2
2,20201015,zs_54_01,5456,2300788,300.0,67970,3
3,20201015,zs_54_01,5456,2300788,1780.0,67972,2
4,20201015,zs_54_01,5456,2300788,1880.0,67973,2
...,...,...,...,...,...,...,...
1002,20201125,zs_96_08,6282,1603317,1997091.0,6817619,2
1003,20201125,zs_96_08,6282,1603515,324319.0,6817660,2
1004,20201125,zs_96_08,6282,1603515,1312687.0,6817675,2
1005,20201125,zt_88_06,8854,2300724,2638915.0,6846290,2


accCode
5222        8
5225       11
5226        6
5269        6
5273       10
5275      106
5276        6
5281        3
5284        5
5287        5
5289        4
5290        4
5291        5
5328        3
5456      143
5470        6
5474        4
6237       88
6282       90
6480        2
6623        3
6627        3
6631        3
6634        7
6683        4
8833        1
8854       45
8865        7
8886        8
9441       27
9451        7
9454        5
9461       17
9471        6
9551        3
9561        1
9685        3
9741        6
9758      265
522201      3
522401      3
522501      1
522601      1
526901      4
527301      3
527501      3
527601      3
527701      7
528101      4
528401      3
528701      4
528901      4
529001      3
529101      3
892402      1
966301      3
966501      3
966701      7
968501      5
974101      3
Name: date, dtype: int64

99.66% SZE orders triggered by msg data
3. IPO stocks selling (ars = 301, 302)
        ApplSeqNum aaa  absFilledThisUpdate  absOrderSize  \
849801         0.0   0                    0         10999   
849802         0.0  -1                    0         10999   
849803         0.0  -1                 5400         10999   
849804         0.0  -1                  300         10999   
849805         0.0  -1                  300         10999   
...            ...  ..                  ...           ...   
302771         0.0   0                    0          1167   
302772         0.0  -1                    0          1167   
302773         0.0  -1                  292          1167   
302774         0.0  -1                  875          1167   
849860         0.0   0                    0          8555   

        absOrderSizeCumFilled  accCode    ars  caamd  cancellationPending  \
849801                      0     5470  302.0      0                  0.0   
849802                      0     

5. updateType 6 orders
There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
         date  accCode  order_x  order_y      prob
30   20201027     6282      234     1610  0.145342
109  20201113     6282      154     1888  0.081568
6. CYB stocks order size < 30w
7. unexpected updateType


MemoryError: 

In [ ]:
print('SZE')
totalAccount = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['colo_broker'] == 'zs_52')]['accCode'].unique()
for date, dateLog in orderLog[(orderLog['secid'] >= 2000000) & (orderLog['colo_broker'] == 'zs_52')].groupby(['date']):
    dailyAccount = dateLog['accCode'].unique()
    print(date, sorted(list(set(totalAccount) - set(dailyAccount))))
print(totalAccount)
print('SSE')
totalAccount = orderLog[(orderLog['secid'] < 2000000) & (orderLog['colo_broker'] == 'zt_52')]['accCode'].unique()
for date, dateLog in orderLog[(orderLog['secid'] < 2000000) & (orderLog['colo_broker'] == 'zt_52')].groupby(['date']):
    dailyAccount = dateLog['accCode'].unique()
    print(date, sorted(list(set(totalAccount) - set(dailyAccount))))
print(totalAccount)

In [ ]:
checkLog = orderLog[(orderLog["updateType"] == 0) & (orderLog['colo_broker'].isin(['zs_52', 'zt_52']))]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
SZE = checkLog[checkLog['secid'] >= 2000000]
SSE = checkLog[checkLog['secid'] < 2000000]
SZE["exchange"] = "SZ"
SSE["exchange"] = "SH"
SZE = SZE[SZE["isMsg"] == 1]
re1 = SSE[(SSE["sta"] == 'sta300')].groupby('date')['internal_latency'].median().reset_index()
re2 = SSE[(SSE["sta"] == 'statwo')].groupby('date')['internal_latency'].median().reset_index()
re3 = SSE[(SSE['sta'] == 'staone')].groupby('date')['internal_latency'].median().reset_index()
re4 = SZE[(SZE["sta"] == 'sta300')].groupby('date')['internal_latency'].median().reset_index()
re5 = SZE[(SZE["sta"] == 'statwo')].groupby('date')['internal_latency'].median().reset_index()
re6 = SZE[(SZE['sta'] == 'staone')].groupby('date')['internal_latency'].median().reset_index()

fig = plt.figure(figsize=(20, 12))
ax = fig.add_subplot(111)
ax.plot(re1['internal_latency'].values, label='sta300 zt52 orders')
ax.plot(re2['internal_latency'].values, label='statwo zt52 orders')
ax.plot(re3['internal_latency'].values, label='staone zs52 orders')
ax.plot(re4['internal_latency'].values, label='sta300 zs52 orders', linestyle='dashed')
ax.plot(re5['internal_latency'].values, label='statwo zs52 orders', linestyle='dashed')
ax.plot(re6['internal_latency'].values, label='staone zs52 orders', linestyle='dashed')

ax.set_title('internal latency median 1015-1125')
indexLs = [i for i in range(re1.shape[0])]
dateLs = [int(i) for i in re1['date'].values]
ax.set_xticks(indexLs)
ax.set_xticklabels(dateLs)
fig.legend(loc=1, bbox_to_anchor=(1,1), bbox_transform=ax.transAxes)
plt.axvline(x=1,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=4,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=5,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=8,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=9,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=12,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=13,ls="--",c="grey", alpha=0.2, lw=1)
bbox = dict(boxstyle="round", fc="0.8")
arrowprops = dict(
    arrowstyle = "->")
ax.annotate('5287,528701 added', (145, 660), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5286 added', (315, 660), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5289, 5290, 528901, 529001 added, 527101 removed', (370, 630), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5291, 529101 added', (540, 600), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5222, 522201 added', (595, 660), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('522401 added', (765, 660), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5225, 5226, 522501, 522601 added', (820, 550), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)

plt.show()

In [ ]:
checkLog = orderLog[(orderLog["updateType"] == 0) & (orderLog['colo_broker'].isin(['zs_52', 'zt_52']))]
checkLog = checkLog[checkLog['caamd'] != 0]
checkLog['internal_latency'] = checkLog["clockAtArrival"] - checkLog["caamd"]
SZE = checkLog[checkLog['secid'] >= 2000000]
SSE = checkLog[checkLog['secid'] < 2000000]
SZE["exchange"] = "SZ"
SSE["exchange"] = "SH"
SZE = SZE[SZE["isMsg"] == 1]
re1 = SSE[(SSE["sta"] == 'sta300')].groupby('date')['internal_latency'].quantile(.95).reset_index()
re2 = SSE[(SSE["sta"] == 'statwo')].groupby('date')['internal_latency'].quantile(.95).reset_index()
re3 = SSE[(SSE['sta'] == 'staone')].groupby('date')['internal_latency'].quantile(.95).reset_index()
re4 = SZE[(SZE["sta"] == 'sta300')].groupby('date')['internal_latency'].quantile(.95).reset_index()
re5 = SZE[(SZE["sta"] == 'statwo')].groupby('date')['internal_latency'].quantile(.95).reset_index()
re6 = SZE[(SZE['sta'] == 'staone')].groupby('date')['internal_latency'].quantile(.95).reset_index()

fig = plt.figure(figsize=(20, 12))
ax = fig.add_subplot(111)
ax.plot(re1['internal_latency'].values, label='sta300 zt52 orders')
ax.plot(re2['internal_latency'].values, label='statwo zt52 orders')
ax.plot(re3['internal_latency'].values, label='staone zs52 orders')
ax.plot(re4['internal_latency'].values, label='sta300 zs52 orders', linestyle='dashed')
ax.plot(re5['internal_latency'].values, label='statwo zs52 orders', linestyle='dashed')
ax.plot(re6['internal_latency'].values, label='staone zs52 orders', linestyle='dashed')

ax.set_title('internal latency 95 percentile 1015-1125')
indexLs = [i for i in range(re1.shape[0])]
dateLs = [int(i) for i in re1['date'].values]
ax.set_xticks(indexLs)
ax.set_xticklabels(dateLs)
fig.legend(loc=1, bbox_to_anchor=(1,1), bbox_transform=ax.transAxes)
plt.axvline(x=1,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=4,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=5,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=8,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=9,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=12,ls="--",c="grey", alpha=0.2, lw=1)
plt.axvline(x=13,ls="--",c="grey", alpha=0.2, lw=1)
bbox = dict(boxstyle="round", fc="0.8")
arrowprops = dict(
    arrowstyle = "->")
ax.annotate('5287,528701 added', (145, 660), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5286 added', (315, 660), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5289, 5290, 528901, 529001 added, 527101 removed', (370, 630), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5291, 529101 added', (540, 600), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5222, 522201 added', (595, 660), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('522401 added', (765, 660), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)
ax.annotate('5225, 5226, 522501, 522601 added', (820, 550), xytext=(20, 0), xycoords='figure pixels', textcoords='offset points',
            bbox=bbox, arrowprops=arrowprops)

plt.show()